<h2 align=center> Toxic Comments Classification using 1D CNN with Keras</h2>

### Import Packages and Functions


In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
#from sklearn.model_selection import train_test_split
print(tf.__version__)

ImportError: No module named tensorflow

### Task 2: Load and Explore Data

In [2]:
train_df = pd.read_csv('train.csv').fillna(' ')
train_df.sample(10,random_state = 1)

SyntaxError: invalid syntax (<ipython-input-2-e25651bac530>, line 1)

In [ ]:
# View few toxic comments


In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
comments = train_df['comment_text'].loc[train_df['toxic']==1].values
wordcloud = WordCloud(
    width = 640,
    height = 640,
    background_color = 'black',
    stopwords = STOPWORDS).generate(str(comments))
fig = plt.figure(
    figsize = (12, 8),
    facecolor = 'k',
    edgecolor = 'k')
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
# Plot frequency of toxic comments


### Task 3: Data Prep — Tokenize and Pad Text Data

In [10]:
max_features = 20000
max_text_length = 400

x_tokenizer = text.Tokenizer(max_features)
x_tokenizer.fit_on_texts(list(x))
x_tokenized  = x_tokenizer.texts_to_sequences(x)
x_train_val = sequence.pad_sequences(x_tokenized , maxlen = max_text_length)


NameError: name 'text' is not defined

### Task 4: Prepare Embedding Matrix with Pre-trained GloVe Embeddings

In [ ]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip -q glove.6B.zip

embedding_dim  = 100
embedding_index = dict()
f = open('glove.6B.100.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:] , dtype = 'float32')
    embeddings_index[word] = coefs
f.close()
print(f'Found {len(embeddings_index)} word vectors. ')


In [ ]:
embedding_matrix = np.zeros((max_features , embedding_dim))
for word , index in x_tokenizer.word_index.items():
    if index > max_features -1:
        break
    else: 
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
            

### Task 5: Create the Embedding Layer

In [ ]:
model = Sequential()
model.add(Embedding(max_features,
                   embedding_dim,
                   embedding_intializer = tf.keras.intializers.Constant(embedding_matrix),
                   trainable=False))



model.add(Dropout(0.2))

### Task 6: Build the Model

In [ ]:
filters =250
kernel_size = 3
hidden_dims = 250

In [ ]:
model.add(Conv1D(filters,
                kernel_size,
                padding = 'valid'))
model.add(MaxPooling1D())
model.add(Conv1D(filters,
                5,
                padding = 'valid',
                activation = 'relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims,activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(1 , activaion ='sigmoid'))
model.summary()

In [ ]:
model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])

### Task 7: Train Model

In [ ]:
x_train , x_val , y_train ,y_val train_test_split(x_train_val ,y,
                                                 test_size = 0.15 ,random_state =1)

%%time
batch_size =32
epochs =3

model.fit(x_train,y_train,batch_size = batch_size,
          epochs =3,
          validation_data = (x_val ,y_val))

### Task 8: Evaluate Model

In [ ]:
test_df = pd.read_csv('test.csv')

In [ ]:
x_test = test_df['comment_text'].values

In [ ]:
x_test_tokenized = x_tokenizer.texts_to_sequences(x_test)
x_testing = sequence.pad_sequences(x_test_tokenized , maxlen = max_text_length)


In [ ]:
y_testing = model.predict(x_testing , verbose =1 ,batch_size =32)

In [ ]:
y_testing.shape

In [ ]:
y_testing[0]

In [ ]:
test_df['Toxic']= ['not toxic' if x < .5 else 'toxic' for x in y_testing]
test_df[['comment_text','Toxic']].head(20) #sample 20